In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

### Pretrain datasets

In [2]:
%%time

import tqdm, torch

df_trans = pq.read_table('data/trans_filtered.pq').to_pandas()

cols = ['url_host', 'request_cnt', 'part_of_day', 'event_time']

for col in tqdm.tqdm(cols):
    df_trans[col] = df_trans[col].apply(torch.tensor)

100%|█████████████████████████████████████████████████████████████████████████████████| 4/4 [00:30<00:00,  7.75s/it]

CPU times: user 42.6 s, sys: 54.2 s, total: 1min 36s
Wall time: 1min 11s


In [3]:
from sklearn.model_selection import train_test_split

df_train_trans, df_valid_trans = train_test_split(df_trans, test_size = 0.1, random_state = 42)
df_train_trans = df_train_trans.to_dict(orient='records')
df_valid_trans = df_valid_trans.to_dict(orient='records')

In [4]:
len(df_train_trans), len(df_valid_trans)

(373785, 41532)

## Train COLES

### Model definition

In [2]:
import torch
import pytorch_lightning as pl
from functools import partial
from ptls.nn import TrxEncoder, RnnSeqEncoder
from ptls.frames.coles import CoLESModule

trx_encoder_params = dict(
    embeddings_noise=0.003,
    numeric_values={'request_cnt': 'identity'
    },
    embeddings={
        'part_of_day': {'in': 4, 'out': 1},
        'url_host': {'in': 132025, 'out': 512}
    }
)


seq_encoder = RnnSeqEncoder(
    trx_encoder=TrxEncoder(**trx_encoder_params),
    hidden_size=512,
    type='gru',
)

model = CoLESModule(
    seq_encoder=seq_encoder,
    optimizer_partial=partial(torch.optim.Adam, lr=0.001),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=10, gamma=0.9),
)

### Dataloader

In [11]:
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing.feature_filter import FeatureFilter
from ptls.frames.coles import ColesDataset
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.frames import PtlsDataModule
from ptls.data_load.augmentations import AllTimeShuffle, DropoutTrx
from  ptls.data_load.datasets import AugmentationDataset



train_ds = MemoryMapDataset(data=df_train_trans)
valid_ds = MemoryMapDataset(data=df_valid_trans)

train_ds = AugmentationDataset(train_ds, f_augmentations = [AllTimeShuffle(), DropoutTrx(trx_dropout=0.01)])

train_data=ColesDataset(train_ds, splitter = SampleSlices(split_count=5, cnt_min=20, cnt_max=200))
valid_data=ColesDataset(valid_ds, splitter = SampleSlices(split_count=5, cnt_min=20, cnt_max=200))

dl = PtlsDataModule(
    train_data=train_data, train_num_workers=16, train_batch_size=256, 
    valid_data=valid_data, valid_num_workers=16, valid_batch_size=256
)

### Trainer

In [12]:
import torch
import pytorch_lightning as pl

import logging

trainer = pl.Trainer(
    max_epochs=50,
    limit_val_batches=100,
    gpus=[1],
    enable_progress_bar=False,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
print(f'logger.version = {trainer.logger.version}')
trainer.fit(model, dl)
print(trainer.logged_metrics)

{'loss': tensor(152.7899, device='cuda:1'), 'seq_len': tensor(85.6484, device='cuda:1'), 'recall_top_k': tensor(0.7711, device='cuda:1')}


/home/morlov/.local/share/virtualenvs/pytorch-lifestream-1iBTwtzi/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [14]:
torch.save(model.state_dict(), "coles-emb-shuffle.pt")

# Inference

In [3]:
model.load_state_dict(torch.load("coles-emb-shuffle.pt"))

<All keys matched successfully>

In [4]:
%%time

import tqdm, torch

df_trans = pq.read_table('data/trans_filtered.pq').to_pandas()

cols = ['url_host', 'request_cnt', 'part_of_day', 'event_time']

for col in tqdm.tqdm(cols):
    df_trans[col] = df_trans[col].apply(torch.tensor)

100%|█████████████████████████████████████████████████████████████████████████████████| 4/4 [00:25<00:00,  6.30s/it]

CPU times: user 41.9 s, sys: 49 s, total: 1min 30s
Wall time: 1min 3s


In [5]:
%%time
import tqdm
from ptls.data_load.datasets import inference_data_loader
import numpy as np

def pooling_inference(model, dl, device='cuda:1'):
    
    model.to(device)
    X = []
    for batch in tqdm.tqdm(dl):
        with torch.no_grad():
            x = model.seq_encoder.trx_encoder(batch.to(device)).payload
            out_max = torch.max(x, dim=1)[0]
            out_min = torch.min(x, dim=1)[0]
            out_mean = torch.mean(x, dim=1)
            out_std = torch.std(x, dim=1)
            features = torch.cat([out_max, out_min, out_mean, out_std], dim=1)      
            X += [features]
    return X

def embed_inference(model, dl, device='cuda:1'):
    
    model.to(device)
    X = []
    for batch in tqdm.tqdm(dl):
        with torch.no_grad():
            features = model.seq_encoder(batch.to(device))
            X += [features]
    return X

dl = inference_data_loader(df_trans.to_dict(orient='records'), num_workers=0, batch_size=128)
X_coles = torch.vstack(embed_inference(model, dl, )).cpu().numpy()
X_pool = torch.vstack(pooling_inference(model, dl, )).cpu().numpy()
X_embeds = np.concatenate([X_coles, X_pool], axis=1)


df_embeds = pd.DataFrame(X_embeds, columns=[f"embed_{e}" for e in range(X_embeds.shape[1])])
df_embeds['user_id'] = df_trans['user_id']
df_embeds.to_csv('./data/coles_512_shuffle.csv', index=False)

3245it [19:01,  2.84it/s]
3245it [01:40, 32.40it/s]


CPU times: user 49min 4s, sys: 6min 4s, total: 55min 9s
Wall time: 33min 17s


## Downstream

In [6]:
%%time

import bisect
import numpy as np

df_embeds = pd.read_csv('./data/coles_512_shuffle.csv')
df_public = pq.read_table('data/public_train.pqt').to_pandas().sort_values(by='user_id')

def age_bucket(x):
    return bisect.bisect_left([18,25,35,45,55,65], x)

y_age = df_public['age']
y_age = np.array(list(map(age_bucket, y_age)))
y_gender = np.array(df_public['is_male'])

X = df_public
X = X.merge(df_embeds, on="user_id", how='inner')
del X['user_id'], X['age'], X['is_male']

CPU times: user 2min 39s, sys: 25.2 s, total: 3min 4s
Wall time: 3min 4s


## Gender

In [7]:
# %%time

from catboost import CatBoostClassifier, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
not_na_gender = (y_gender != 'NA') & (y_gender != None)
x_train, x_test_gender, y_train, y_test_gender = train_test_split(X[not_na_gender], y_gender[not_na_gender], test_size = 0.1, random_state = 42)

clf_gender = CatBoostClassifier(
    iterations=1000,
    custom_metric=[metrics.AUC()],
    use_best_model=True,
    random_seed=42)
clf_gender.fit(x_train, y_train, metric_period=100, eval_set=(x_test_gender, y_test_gender))

Learning rate set to 0.122426
0:	learn: 0.6459587	test: 0.6453993	best: 0.6453993 (0)	total: 202ms	remaining: 3m 22s
100:	learn: 0.4638823	test: 0.4658462	best: 0.4658462 (100)	total: 13.3s	remaining: 1m 58s
200:	learn: 0.4473522	test: 0.4568446	best: 0.4568446 (200)	total: 26.1s	remaining: 1m 43s
300:	learn: 0.4356581	test: 0.4535524	best: 0.4535524 (300)	total: 38.8s	remaining: 1m 30s
400:	learn: 0.4255570	test: 0.4515665	best: 0.4515665 (400)	total: 51.6s	remaining: 1m 17s
500:	learn: 0.4165624	test: 0.4505126	best: 0.4505126 (500)	total: 1m 4s	remaining: 1m 4s
600:	learn: 0.4080322	test: 0.4499383	best: 0.4499383 (600)	total: 1m 16s	remaining: 51.1s
700:	learn: 0.3998937	test: 0.4492137	best: 0.4492137 (700)	total: 1m 29s	remaining: 38.3s
800:	learn: 0.3922492	test: 0.4487837	best: 0.4487837 (800)	total: 1m 42s	remaining: 25.5s
900:	learn: 0.3846440	test: 0.4486591	best: 0.4486591 (900)	total: 1m 55s	remaining: 12.7s
999:	learn: 0.3776097	test: 0.4482599	best: 0.4482599 (999)	total

In [12]:
print(f'GINI по полу {2 * roc_auc_score(y_test_gender, clf_gender.predict_proba(x_test_gender)[:,1]) - 1:2.3f}')

GINI по полу 0.744


# Age

In [9]:
%%time

from sklearn.metrics import classification_report

not_na_age = ~np.isnan(y_age)
x_train, x_test_age, y_train, y_test_age = train_test_split(X[not_na_age], y_age[not_na_age], test_size = 0.1, random_state = 42)

clf_age = CatBoostClassifier(iterations=1000,
    custom_metric=[metrics.Accuracy()],
    use_best_model=True,
    random_seed=42)
clf_age.fit(x_train, y_train, metric_period=100, eval_set=(x_test_age, y_test_age))

Learning rate set to 0.120515
0:	learn: 1.8312447	test: 1.8296243	best: 1.8296243 (0)	total: 985ms	remaining: 16m 23s
100:	learn: 1.2915728	test: 1.2932410	best: 1.2932410 (100)	total: 1m 29s	remaining: 13m 15s
200:	learn: 1.2591813	test: 1.2754656	best: 1.2754656 (200)	total: 2m 52s	remaining: 11m 25s
300:	learn: 1.2370398	test: 1.2680157	best: 1.2680157 (300)	total: 4m 14s	remaining: 9m 50s
400:	learn: 1.2188216	test: 1.2641718	best: 1.2641718 (400)	total: 5m 34s	remaining: 8m 19s
500:	learn: 1.2023945	test: 1.2614653	best: 1.2614653 (500)	total: 6m 54s	remaining: 6m 52s
600:	learn: 1.1873805	test: 1.2600156	best: 1.2600156 (600)	total: 8m 18s	remaining: 5m 30s
700:	learn: 1.1723051	test: 1.2584579	best: 1.2584579 (700)	total: 9m 42s	remaining: 4m 8s
800:	learn: 1.1582417	test: 1.2573333	best: 1.2573333 (800)	total: 11m 5s	remaining: 2m 45s
900:	learn: 1.1448817	test: 1.2563728	best: 1.2563728 (900)	total: 12m 27s	remaining: 1m 22s
999:	learn: 1.1318097	test: 1.2553813	best: 1.255381

In [13]:
print(classification_report(y_test_age, clf_age.predict(x_test_age), \
                            target_names = ['<18', '18-25','25-34', '35-44', '45-54', '55-65', '65+']))

              precision    recall  f1-score   support

         <18       0.00      0.00      0.00       109
       18-25       0.55      0.39      0.45      3238
       25-34       0.52      0.64      0.57      8863
       35-44       0.42      0.51      0.46      7773
       45-54       0.39      0.26      0.31      4218
       55-65       0.40      0.26      0.32      2254
         65+       0.29      0.02      0.03       545

    accuracy                           0.47     27000
   macro avg       0.37      0.30      0.31     27000
weighted avg       0.46      0.47      0.45     27000



/home/morlov/.local/share/virtualenvs/pytorch-lifestream-1iBTwtzi/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/morlov/.local/share/virtualenvs/pytorch-lifestream-1iBTwtzi/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/morlov/.local/share/virtualenvs/pytorch-lifestream-1iBTwtzi/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_div

In [11]:
0.752 + 2*0.46

1.6720000000000002